## Circuit cutting with automatic cut finding using the Circuit Knitting Toolbox

### Import relevant modules

In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Options,
    Session,
    Sampler,
    RuntimeOptions,
)

from circuit_knitting_toolbox.circuit_cutting import WireCutter

### Create a circuit to cut

In [2]:
qc = QuantumCircuit(5)
for i in range(5):
    qc.h(i)
qc.cx(0, 1)
for i in range(2, 5):
    qc.t(i)
qc.cx(0, 2)
qc.rx(np.pi / 2, 4)
qc.rx(np.pi / 2, 0)
qc.rx(np.pi / 2, 1)
qc.cx(2, 4)
qc.t(0)
qc.t(1)
qc.cx(2, 3)
qc.ry(np.pi / 2, 4)
for i in range(5):
    qc.h(i)

qc.draw()

┌───┐                ┌─────────┐   ┌───┐   ┌───┐
q_0: ┤ H ├──■───────■─────┤ Rx(π/2) ├───┤ T ├───┤ H ├
     ├───┤┌─┴─┐     │     ├─────────┤   ├───┤   ├───┤
q_1: ┤ H ├┤ X ├─────┼─────┤ Rx(π/2) ├───┤ T ├───┤ H ├
     ├───┤├───┤   ┌─┴─┐   └─────────┘   └───┘   ├───┤
q_2: ┤ H ├┤ T ├───┤ X ├────────■──────────■─────┤ H ├
     ├───┤├───┤   └───┘        │        ┌─┴─┐   ├───┤
q_3: ┤ H ├┤ T ├────────────────┼────────┤ X ├───┤ H ├
     ├───┤├───┤┌─────────┐   ┌─┴─┐   ┌──┴───┴──┐├───┤
q_4: ┤ H ├┤ T ├┤ Rx(π/2) ├───┤ X ├───┤ Ry(π/2) ├┤ H ├
     └───┘└───┘└─────────┘   └───┘   └─────────┘└───┘

### Set up the Qiskit runtime service

In [5]:
service_args = QiskitRuntimeService(
    channel="ibm_quantum",
    token="<YOUR_API_KEY>",
).active_account()

### Find cuts that match our criteria

In [6]:
# Set the Sampler and runtime options
options = Options(resilience_level=1, optimization_level=3, execution={"shots": 8192})
runtime_options = RuntimeOptions(backend="ibmq_qasm_simulator")

# Instantiate a WireCutter and decompose the circuit
cutter = WireCutter(
    qc, service_args=service_args, options=options, runtime_options=runtime_options
)
# cutter = WireCutter(qc) # local Estimator

cuts = cutter.decompose(
    method="automatic",
    max_subcircuit_width=6,
    max_cuts=2,
    num_subcircuits=[2],
)

(_cut_automatic pid=10207) Exporting as a LP file to let you check the model that will be solved :  inf <class 'float'>
(_cut_automatic pid=10207) Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
(_cut_automatic pid=10207) CPXPARAM_Read_DataCheck                          1
(_cut_automatic pid=10207) CPXPARAM_TimeLimit                               300
(_cut_automatic pid=10207) Warning:  Non-integral bounds for integer variables rounded.
(_cut_automatic pid=10207) Tried aggregator 3 times.
(_cut_automatic pid=10207) MIP Presolve eliminated 20 rows and 5 columns.
(_cut_automatic pid=10207) MIP Presolve modified 4 coefficients.
(_cut_automatic pid=10207) Aggregator did 24 substitutions.
(_cut_automatic pid=10207) Reduced MIP has 44 rows, 19 columns, and 131 nonzeros.
(_cut_automatic pid=10207) Reduced MIP has 15 binaries, 4 generals, 0 SOSs, and 0 indicators.
(_cut_automatic pid=10207) Presolve time = 0.00 sec. (0.31 ticks)
(_cut_automatic pid=10207) Found incumbent of value 1.000000

### Evaluate the subcircuits, then recompose the circuit and verify the error between the full and cut circuit distributions is within tolerance

In [7]:
# Evaluate the subcircuits on backend
subcircuit_instance_probabilities = cutter.evaluate(cuts)

# Recompose the circuit and generate the cut circuit's probability distribution
reconstructed_probabilities = cutter.recompose(
    subcircuit_instance_probabilities, cuts, num_threads=4
)

# Use a statevector simulator to calculate the error between the inferred and actual distributions
metrics = cutter.verify(reconstructed_probabilities)
print(metrics)

{'nearest': {'chi2': 0.0022086910955534155, 'Mean Squared Error': 5.154099340326516e-06, 'Mean Absolute Percentage Error': 8.251430447297219, 'Cross Entropy': 2.60191073304632, 'HOP': 0.8991855755448341}, 'naive': {'chi2': 0.0022086910955534155, 'Mean Squared Error': 5.154099340326516e-06, 'Mean Absolute Percentage Error': 8.251430447297219, 'Cross Entropy': 2.60191073304632, 'HOP': 0.8991855755448341}}
